In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data_feature.csv")
df

,frame,cls,r_mean,g_mean,b_mean,r_range,g_range,b_range,r_var,g_var,...,g_skewness,b_skewness,r_kurtosis,g_kurtosis,b_kurtosis,luminance,xmin,xmax,ymin,ymax
0,10803,0,64.928267,58.074811,61.582150,249,255,254,2355.670596,2500.595571,...,1.196594,0.990761,1.001988,0.638622,0.117479,59.784049,105,281,238,310
1,10803,0,138.997637,124.954700,114.533424,255,255,255,5666.912828,5518.423918,...,-0.152255,-0.159831,-1.404527,-1.323314,-1.094660,127.187450,356,480,282,538
2,10803,0,97.098447,88.644178,86.850476,252,255,255,4424.019261,4590.634355,...,0.841100,0.714087,-0.304263,-0.349028,-0.557364,90.311384,281,625,203,352
3,10803,0,107.192779,91.544487,88.218246,209,192,193,2574.915770,2562.648408,...,0.239058,0.206782,-1.242052,-1.313084,-1.232371,94.629927,1089,1122,148,242
4,10803,0,105.371599,96.455215,96.100057,224,236,238,1415.344057,1460.645954,...,0.804296,0.731621,0.029867,0.521455,0.768797,98.324340,1052,1088,146,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,11016,0,86.684028,70.076389,64.145833,193,181,181,3059.875856,2850.542776,...,1.224627,1.210468,0.025189,0.005563,0.028951,73.177919,945,961,154,190
724,11016,0,102.333998,93.595966,92.582890,247,244,239,1645.663933,1882.914106,...,0.956812,1.096345,0.374047,0.554085,1.084402,95.379755,1032,1079,146,242
725,11016,0,119.870258,113.127946,109.607969,242,250,254,4990.736029,5148.588344,...,0.395233,0.325274,-1.165831,-1.070169,-1.049301,114.306897,694,748,210,375
726,11016,0,115.791870,105.113008,95.369106,131,143,138,1241.403836,1303.241701,...,-0.043394,-0.003271,-1.309186,-1.316035,-1.288657,106.679731,885,915,247,288


In [25]:
df.iloc[0]['xmin']

105.0

In [19]:
type(df[df['frame']==10803].iloc[3,:])

pandas.core.series.Series

In [38]:
def if_occlusion(df: pd.DataFrame,i:int,j:int)->bool:
    x_min_1 = df.iloc[i]['xmin']
    x_min_2 = df.iloc[j]['xmin']
    x_max_1 = df.iloc[i]['xmax']
    x_max_2 = df.iloc[j]['xmax']
    y_min_1 = df.iloc[i]['xmin']
    y_min_2 = df.iloc[j]['ymin']
    y_max_1 = df.iloc[i]['ymax']
    y_max_2 = df.iloc[j]['ymax']
    if (x_min_2>x_min_1 and x_min_2<x_max_1) or (x_max_2<x_max_1 and x_max_2>x_min_1):
        #when j's left side is inbetween i's width or when j's right side is inbetween i's width
        if (y_min_2<y_max_1 and y_min_2>y_min_1) or (y_max_2>y_min_1 and y_max_2<y_max_1):
            #when j's up side is inbetween i's height or when j's down side is inbetween i's height
            return True
    return False
occlusion_list = []
for frame in df['frame'].unique():
    #for each frame
    new_df = df[df['frame']==frame].copy()
    curr_list = [0]*len(new_df)
    if len(new_df)==1:#if this frame just have a single box, there cannot be occlusion
        occlusion_list.extend(curr_list)
        break
    #when there are multiple boxes:
    for i in range(0,len(new_df)-1):
        for j in range(i+1,len(new_df)):
            if i!=j:
                if if_occlusion(df=new_df,i=i,j=j):
                    curr_list[i] += 1
                    curr_list[j] += 1
    occlusion_list.extend(curr_list)
print(occlusion_list) 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 3, 2, 2, 0, 0, 0, 0, 0, 1, 3, 2, 1, 0, 0, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 1, 3, 2, 0, 0, 0, 0, 0, 1, 1, 2, 1, 4, 1, 0, 0, 0, 0, 0, 1, 2, 2, 4, 1, 0, 0, 0, 0, 0, 0, 2, 2, 1, 3, 0, 0, 0, 0, 0, 0, 1, 1, 1, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 3, 4, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2, 4, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2, 0, 4, 0, 0, 0, 0, 0, 2, 3, 2, 0, 0, 3, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 

In [40]:
df['occlusion'] = occlusion_list

In [42]:
df['occlusion'].unique()

array([0, 1, 2, 3, 4])